# **Methode 1**

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

# Prétraitement du texte
text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Enregistrer le tokenizer
with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")



Taille du vocabulaire : 6745


In [2]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(5, len(sequence_data)):
    words = sequence_data[i-5:i+1]
    sequences.append(words)

sequences = np.array(sequences)



In [3]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  
y = sequences[:, -1]  
y = to_categorical(y, num_classes=vocab_size)





In [4]:
# Définir le modèle
model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(120))
model.add(Dense(vocab_size, activation='softmax'))
model.build(input_shape=(None, X.shape[1])) 
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 5, 50)          │       337,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 120)            │        82,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6745)           │       816,145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,235,475 (4.71 MB)

 Trainable params: 1,235,475 (4.71 MB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=80, verbose=1)



Epoch 1/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 60s 15ms/step - accuracy: 0.0429 - loss: 6.5357
Epoch 2/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 83s 16ms/step - accuracy: 0.1025 - loss: 5.6091
Epoch 3/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 59s 15ms/step - accuracy: 0.1325 - loss: 5.1712
Epoch 4/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 59s 15ms/step - accuracy: 0.1520 - loss: 4.8798
Epoch 5/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 15ms/step - accuracy: 0.1657 - loss: 4.6363
Epoch 6/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 15ms/step - accuracy: 0.1766 - loss: 4.4277
Epoch 7/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 15ms/step - accuracy: 0.1931 - loss: 4.2247
Epoch 8/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 15ms/step - accuracy: 0.2097 - loss: 4.0311
Epoch 9/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 85s 16ms/step - accuracy: 0.2277 - loss: 3.8566
Epoch 10/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 61s 16ms/step - accuracy: 0.2495 - loss: 3.6940
Epoch 11/80
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 82s 16ms/step - accuracy: 0.2706 - loss: 3.5390
Epoch 12

In [6]:
input_text = "i want to"
predict_next_words = 45


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-5:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

i want to have been particular to other in the course of the evening it ought to be a sensible young man shan’t for all very i told no more of the certainty for seeing you am well cried miss bennet i shall remember a sense of delight


# **Methode 2**

In [7]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense

In [8]:
# my_input_sequences = []
# for line in text.split('\n'):
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         my_n_gram_sequence = token_list[:i+1]
#         my_input_sequences.append(my_n_gram_sequence)


In [9]:
# max_sequence_len = max([len(seq) for seq in my_input_sequences])
# input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
# X = input_sequences[:, :-1]
# y = input_sequences[:, -1]

# y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [11]:
# model = Sequential()
# model.add(Embedding(total_words, 60, input_length=max_sequence_len-1))
# model.add(LSTM(150))
# model.add(Dense(total_words, activation='softmax'))
# print(model.summary())

In [12]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X, y, epochs=50, verbose=1)

In [13]:
# input_text = "Joe biden"
# predict_next_words= 6

# for _ in range(predict_next_words):
#     token_list = tokenizer.texts_to_sequences([input_text])[0]
#     token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#     predicted = np.argmax(model.predict(token_list), axis=-1)
#     output_word = ""
#     for word, index in tokenizer.word_index.items():
#         if index == predicted:
#             output_word = word
#             break
#     input_text += " " + output_word

# print(input_text)